<div class="alert alert-danger">



<B><font size='80'><center>STOP HERE... ABORT!</center></font></b>
<center>This was my initial Wrangle file.</center>
* It is only here for review and future research
* This data pull was not used because of the extraneous amount of Null/NaN in the data.

# ```02_wrangle.ipynb``` was used for the project.

# Imports

In [1]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
from xml.etree import ElementTree

import matplotlib.pyplot as plt
# Only works inside notebook
%matplotlib inline 
#import matplotlib as mpl
#mpl.rcParams['agg.path.chunksize'] = 10000 # assists with processor speed

# import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer

from sklearn.neighbors import KNeighborsClassifier

# import helper files from local environment
from env import user, password, host
import QMCBT_00_quicktips as qt
import QMCBT_01_acquire as acquire
import QMCBT_02_prepare as prepare
import QMCBT_03_explore as explore
import QMCBT_04_model as model
import QMCBT_05_evaluate as evaluate
import QMCBT_explore_evaluate as ee
import QMCBT_wrangle as w

# allows import reload without needing to clear kernel and rerun
from importlib import reload
# reload(packagename) 

import warnings
warnings.filterwarnings("ignore")

**CUSTOM EXPLORATION FUNCTIONS
nunique_column_all(df): PRINT NUNIQUE OF ALL COLUMNS
nunique_column_objects(df): PRINT NUNIQUE OF COLUMNS THAT ARE OBJECTS
nunique_column_qty(df): PRINT NUNIQUE OF COLUMNS THAT ARE *NOT* OBJECTS
numeric_range(df): COMPUTE RANGE FOR ALL NUMERIC VARIABLES

**USEFUL EXPLORATORY CODE**
DFNAME.head()
DFNAME.shape
DFNAME.shape[0] #read row count
DFNAME.describe().T
DFNAME.columns.to_list()
DFNAME.COLUMNNAME.value_counts(dropna=False)
DFNAME.dtypes
DFNAME.select_dtypes(include='object').columns
DFNAME.select_dtypes(include='float').columns
pd.crosstab(DFNAME.COLUMN-1, DFNAME.COLUMN-2)


In [2]:
# Left Align Tables in Jupyter Notebook
from IPython.core.display import HTML
table_css = 'table {align:left;display:block}'
HTML('<style>{}</style>'.format(table_css))

<div class="alert alert-success">

# Life Expectancy & Mortality across the Globe
* **GHO Selections:** https://apps.who.int/gho/athena/api/GHO
* **Homepage:** https://www.who.int/data/gho/info/athena-api-examples
* **CSV Data Pull:** https://apps.who.int/gho/athena/api/GHO/WHOSIS_000001,WHOSIS_000002,WHOSIS_000003,WHOSIS_000004,WHOSIS_000005,WHOSIS_000007,WHOSIS_000008,WHOSIS_000009,WHOSIS_000010,WHOSIS_000012,WHOSIS_000013,WHOSIS_000014,WHOSIS_000015,WHOSIS_000016,MDG_0000000001,MDG_0000000003,MDG_0000000007,MDG_0000000025,MDG_0000000026?format=csv
    
<div class="alert alert-danger">

### Due to latency and timeout issues, the ```CSV Data Pull``` weblink above must be used to pull the CSV directly and save to a local file. Multiple refresh attempts may be required before getting the save file prompt.

In [3]:
# Read in csv from local file as leam_df (Life Expectancy And Mortality DataFrame)
leam_df = pd.read_csv('dirty_leam.csv')

In [4]:
# Check DataFrame
leam_df

,GHO,PUBLISHSTATE,YEAR,REGION,UNREGION,WORLDBANKINCOMEGROUP,COUNTRY,AGEGROUP,SEX,UNSDGREGION,Display Value,Numeric,Low,High,StdErr,StdDev,Comments
0,MDG_0000000026,PUBLISHED,2004,AFR,NaN,NaN,SLE,NaN,NaN,NaN,1 850 [ 1 480 - 2 330 ],1850.00000,1480.00000,2330.00000,NaN,NaN,NaN
1,MDG_0000000026,PUBLISHED,2000,AFR,NaN,NaN,SSD,NaN,NaN,NaN,1 730 [ 1 320 - 2 250 ],1730.00000,1320.00000,2250.00000,NaN,NaN,NaN
2,MDG_0000000026,PUBLISHED,2005,AFR,NaN,NaN,SLE,NaN,NaN,NaN,1 760 [ 1 410 - 2 220 ],1760.00000,1410.00000,2220.00000,NaN,NaN,NaN
3,MDG_0000000026,PUBLISHED,2001,AFR,NaN,NaN,SSD,NaN,NaN,NaN,1 690 [ 1 310 - 2 190 ],1690.00000,1310.00000,2190.00000,NaN,NaN,NaN
4,MDG_0000000026,PUBLISHED,2002,AFR,NaN,NaN,CAF,NaN,NaN,NaN,1 280 [ 872 - 1 880 ],1280.00000,872.00000,1880.00000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122195,MDG_0000000007,PUBLISHED,2016,AFR,NaN,NaN,ZWE,NaN,MLE,NaN,63.56 [54.18-74.12],63.56478,54.18149,74.12002,NaN,NaN,NaN
122196,MDG_0000000007,PUBLISHED,2018,AFR,NaN,NaN,ZWE,NaN,FMLE,NaN,49.82 [39.47-62.87],49.82213,39.46889,62.87403,NaN,NaN,NaN
122197,MDG_0000000007,PUBLISHED,2019,AFR,NaN,NaN,ZWE,NaN,BTSX,NaN,54.24 [41.04-71.54],54.23561,41.04484,71.53995,NaN,NaN,NaN
122198,MDG_0000000025,PUBLISHED,2008-2014,GLOBAL,NaN,NaN,NaN,NaN,NaN,NaN,76.9,76.85271,NaN,NaN,NaN,NaN,UNICEF/WHO joint global database on Skilled at...


In [5]:
# List Columns
leam_df.columns

Index(['GHO', 'PUBLISHSTATE', 'YEAR', 'REGION', 'UNREGION',
       'WORLDBANKINCOMEGROUP', 'COUNTRY', 'AGEGROUP', 'SEX', 'UNSDGREGION',
       'Display Value', 'Numeric', 'Low', 'High', 'StdErr', 'StdDev',
       'Comments'],
      dtype='object')

<div class="alert alert-warning">

#### GHO Stats

In [6]:
# Show unique GHO Codes to ensure proper Data Pull
leam_df.GHO.unique()

array(['MDG_0000000026', 'WHOSIS_000004', 'MDG_0000000003',
       'WHOSIS_000014', 'WHOSIS_000015', 'WHOSIS_000001', 'WHOSIS_000002',
       'WHOSIS_000007', 'MDG_0000000025', 'MDG_0000000001',
       'WHOSIS_000016', 'WHOSIS_000003', 'MDG_0000000007'], dtype=object)

In [7]:
# Check GHO Null count
leam_df.GHO.isnull().sum()

0

In [8]:
# Check GHO value counts
leam_df.GHO.value_counts()

MDG_0000000007    36084
MDG_0000000001    35933
WHOSIS_000004      9690
WHOSIS_000003      9648
WHOSIS_000016      6262
MDG_0000000003     5345
WHOSIS_000014      4353
MDG_0000000026     3420
WHOSIS_000015      2328
WHOSIS_000001      2328
WHOSIS_000002      2328
WHOSIS_000007      2328
MDG_0000000025     2153
Name: GHO, dtype: int64

<div class="alert alert-warning">

#### MDG Stats

<div class="alert alert-info">

#### Over 35K value counts
* 'MDG_0000000007': 'Under-five mortality rate (per 1000 live births)', 
* 'MDG_0000000001': 'Infant mortality rate (between birth and 11 months per 1000 live births)', 

#### Between 2-5k value counts
* 'MDG_0000000003': 'Adolescent birth rate (per 1000 women)', 
* 'MDG_0000000026': 'Maternal mortality ratio (per 100 000 live births)', 
* 'MDG_0000000025': 'Births attended by skilled health personnel (%)', 

<div class="alert alert-danger">
    
### REMOVE FROM DATA PULL
#### MDG...05 returned no data
* 'MDG_0000000005': 'Contraceptive prevalence (%)', 

In [9]:
# Show percentage of entries for MDG GHO Codes

print('Over 35K value counts')
print(f'MDG_0000000007    {round(36084/leam_df.shape[0]*100,2)}%')
print(f'MDG_0000000001    {round(35933/leam_df.shape[0]*100,2)}%')
print()
print('Between 2-5k value counts')
print(f'MDG_0000000003     {round(5345/leam_df.shape[0]*100,2)}%')
print(f'MDG_0000000026     {round(3420/leam_df.shape[0]*100,2)}%')
print(f'MDG_0000000025     {round(2153/leam_df.shape[0]*100,2)}%')

Over 35K value counts
MDG_0000000007    29.53%
MDG_0000000001    29.41%

Between 2-5k value counts
MDG_0000000003     4.37%
MDG_0000000026     2.8%
MDG_0000000025     1.76%


#### MDG Definitions:

* 'MDG_0000000001': 'Infant mortality rate (between birth and 11 months per 1000 live births)', 
* 'MDG_0000000003': 'Adolescent birth rate (per 1000 women)', 
* 'MDG_0000000005': 'Contraceptive prevalence (%)', 
* 'MDG_0000000007': 'Under-five mortality rate (per 1000 live births)', 
* 'MDG_0000000025': 'Births attended by skilled health personnel (%)', 
* 'MDG_0000000026': 'Maternal mortality ratio (per 100 000 live births)', 

#### MDG Data Dictionary

|[GHO Code](https://apps.who.int/gho/athena/api/GHO)      |[Documentation](https://www.who.int/data/gho/indicator-metadata-registry)|[Global Health Observatory](https://www.who.int/data/gho) (GHO) Code Description|
|:-------------|:-----------:|:------------------------------------------------|
|MDG_0000000001|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/1)|Infant mortality rate (between birth and 11 months per 1000 live births)| 
|MDG_0000000003|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/4669)|Adolescent birth rate (per 1000 women)| 
|MDG_0000000007|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/7)|Under-five mortality rate (per 1000 live births)| 
|MDG_0000000025|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/25)|Births attended by skilled health personnel (%)| 
|MDG_0000000026|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/26)|Maternal mortality ratio (per 100 000 live births)| 

<div class="alert alert-warning">

#### WHOSIS Stats

<div class="alert alert-info">

#### Over 9600 value count
* 'WHOSIS_000004': 'Adult mortality rate (probability of dying between 15 and 60 years per 1000 population)',
* 'WHOSIS_000003': 'Neonatal mortality rate (0 to 27 days) per 1000 live births)',

#### Between 2-5k value counts
* 'WHOSIS_000016': 'Mortality rate among children ages 5 to 9 years (per 1000 children aged 5)', 
* 'WHOSIS_000014': 'Stillbirth rate (per 1000 total births)', 

#### These are all exactly the same record count at 2,328 entries
* 'WHOSIS_000001': 'Life expectancy at birth (years)', 
* 'WHOSIS_000002': 'Healthy life expectancy (HALE) at birth (years)', 
* 'WHOSIS_000007': 'Healthy life expectancy (HALE) at age 60 (years)', 
* 'WHOSIS_000015': 'Life expectancy at age 60 (years)', 

<div class="alert alert-danger">
    
### REMOVE FROM DATA PULL
#### Very little representation at only 1.54% with just 615 records 
* 'WHOSIS_000006': 'Infants exclusively breastfed for the first six months of life (%)', 

In [10]:
# Show percentage of entries for WHOSIS GHO Codes

print('Over 9600 value count')
print(f'WHOSIS_000004    {round(9690/leam_df.shape[0]*100,2)}%')
print(f'WHOSIS_000003    {round(9648/leam_df.shape[0]*100,2)}%')
print('')
print('Between 2-5k value counts')
print(f'WHOSIS_000016     {round(6262/leam_df.shape[0]*100,2)}%')
print(f'WHOSIS_000014     {round(4353/leam_df.shape[0]*100,2)}%')
print('')
print('These are all exactly the same record count at 2,328 entries')
print(f'WHOSIS_000001     {round(2328/leam_df.shape[0]*100,2)}%')
print(f'WHOSIS_000002     {round(2328/leam_df.shape[0]*100,2)}%')
print(f'WHOSIS_000007     {round(2328/leam_df.shape[0]*100,2)}%')
print(f'WHOSIS_000015     {round(2328/leam_df.shape[0]*100,2)}%')

Over 9600 value count
WHOSIS_000004    7.93%
WHOSIS_000003    7.9%

Between 2-5k value counts
WHOSIS_000016     5.12%
WHOSIS_000014     3.56%

These are all exactly the same record count at 2,328 entries
WHOSIS_000001     1.91%
WHOSIS_000002     1.91%
WHOSIS_000007     1.91%
WHOSIS_000015     1.91%


#### WHOSIS Definitions:

* 'WHOSIS_000001': 'Life expectancy at birth (years)', 
* 'WHOSIS_000002': 'Healthy life expectancy (HALE) at birth (years)', 
* 'WHOSIS_000003': 'Neonatal mortality rate (0 to 27 days) per 1000 live births) (SDG 3.2.2)',
* 'WHOSIS_000004': 'Adult mortality rate (probability of dying between 15 and 60 years per 1000 population)',
* 'WHOSIS_000006': 'Infants exclusively breastfed for the first six months of life (%)', 
* 'WHOSIS_000007': 'Healthy life expectancy (HALE) at age 60 (years)', 
* 'WHOSIS_000014': 'Stillbirth rate (per 1000 total births)', 
* 'WHOSIS_000015': 'Life expectancy at age 60 (years)', 
* 'WHOSIS_000016': 'Mortality rate among children ages 5 to 9 years (per 1000 children aged 5)', 

#### WHOSIS Data Dictionary

|[GHO Code](https://apps.who.int/gho/athena/api/GHO)      |[Documentation](https://www.who.int/data/gho/indicator-metadata-registry)|[Global Health Observatory](https://www.who.int/data/gho) (GHO) Code Description|
|:-------------|:-----------:|:------------------------------------------------|
|WHOSIS_000001|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/65)|Life expectancy at birth (years)| 
|WHOSIS_000002|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/66)|Healthy life expectancy (HALE) at birth (years)| 
|WHOSIS_000003|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/67)|Neonatal mortality rate (0 to 27 days) per 1000 live births)|
|WHOSIS_000004|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/64)|Adult mortality rate (probability of dying between 15 and 60 years per 1000 population)|
|WHOSIS_000007|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/3443)|Healthy life expectancy (HALE) at age 60 (years)| 
|WHOSIS_000014|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/2444)|Stillbirth rate (per 1000 total births)| 
|WHOSIS_000015|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/2977)|Life expectancy at age 60 (years)| 
|WHOSIS_000016|[📖](https://www.who.int/data/gho/indicator-metadata-registry/imr-details/5669)|Mortality rate among children ages 5 to 9 years (per 1000 children aged 5)| 


<div class="alert alert-warning">

# DROP Columns

In [11]:
# Show Null Count by Column in descending order
leam_df.isnull().sum().sort_values(ascending=False)

StdDev                  122200
StdErr                  122200
UNREGION                122046
WORLDBANKINCOMEGROUP    121948
UNSDGREGION             121780
AGEGROUP                116855
Comments                114642
High                     26500
Low                      26500
SEX                       9926
COUNTRY                   3403
REGION                     798
Display Value                0
Numeric                      0
PUBLISHSTATE                 0
YEAR                         0
GHO                          0
dtype: int64

In [12]:
# Drop all columns with more than 10k Null and uneeded features
leam_df = leam_df.drop(columns=['StdDev', 
                                'StdErr', 
                                'UNREGION', 
                                'UNSDGREGION', 
                                'WORLDBANKINCOMEGROUP', 
                                'AGEGROUP', 
                                'Comments', 
                                'High',
                                'PUBLISHSTATE',
                                'Display Value',
                                'Low'])

In [13]:
# Check work
leam_df

,GHO,YEAR,REGION,COUNTRY,SEX,Numeric
0,MDG_0000000026,2004,AFR,SLE,NaN,1850.00000
1,MDG_0000000026,2000,AFR,SSD,NaN,1730.00000
2,MDG_0000000026,2005,AFR,SLE,NaN,1760.00000
3,MDG_0000000026,2001,AFR,SSD,NaN,1690.00000
4,MDG_0000000026,2002,AFR,CAF,NaN,1280.00000
...,...,...,...,...,...,...
122195,MDG_0000000007,2016,AFR,ZWE,MLE,63.56478
122196,MDG_0000000007,2018,AFR,ZWE,FMLE,49.82213
122197,MDG_0000000007,2019,AFR,ZWE,BTSX,54.23561
122198,MDG_0000000025,2008-2014,GLOBAL,NaN,NaN,76.85271


In [14]:
# Show Null Count by Column in descending order
leam_df.isnull().sum().sort_values(ascending=False)

SEX        9926
COUNTRY    3403
REGION      798
GHO           0
YEAR          0
Numeric       0
dtype: int64

<div class="alert alert-warning">

#### Sex

In [15]:
# Show Null count for SEX
leam_df.SEX.isnull().sum()

9926

In [16]:
# Display Null percentage of entire DataFrame for SEX
print(f'Sex Null is {round(leam_df.SEX.isnull().sum() / leam_df.shape[0] *100, 2)} % of entire database')

Sex Null is 8.12 % of entire database


In [17]:
# Create a mask df for Null SEX
sex_df = leam_df['SEX'].isnull()

In [18]:
# Take a look
leam_df[sex_df]

,GHO,YEAR,REGION,COUNTRY,SEX,Numeric
0,MDG_0000000026,2004,AFR,SLE,NaN,1850.00000
1,MDG_0000000026,2000,AFR,SSD,NaN,1730.00000
2,MDG_0000000026,2005,AFR,SLE,NaN,1760.00000
3,MDG_0000000026,2001,AFR,SSD,NaN,1690.00000
4,MDG_0000000026,2002,AFR,CAF,NaN,1280.00000
...,...,...,...,...,...,...
40435,MDG_0000000026,2014,EUR,ITA,NaN,2.00000
40436,MDG_0000000026,2015,EUR,ITA,NaN,2.00000
121912,MDG_0000000025,2001-2007,GLOBAL,NaN,NaN,64.12033
121913,MDG_0000000025,2015-2021,GLOBAL,NaN,NaN,83.56604


In [19]:
# List unique GHO values with NaN entry for SEX
leam_df[sex_df].GHO.unique()

array(['MDG_0000000026', 'WHOSIS_000014', 'MDG_0000000025'], dtype=object)

In [20]:
# Show value counts for SEX to include NaN
leam_df[sex_df].value_counts(dropna=False)

GHO             YEAR       REGION  COUNTRY  SEX  Numeric  
MDG_0000000025  1999-2000  AFR     LBR      NaN  50.90000     1
WHOSIS_000014   2004       WPR     FSM      NaN  14.40183     1
                           SEAR    THA      NaN  9.15372      1
                                   TLS      NaN  19.21290     1
                           WPR     AUS      NaN  2.86615      1
                                                             ..
MDG_0000000026  2002       EMR     PAK      NaN  264.00000    1
                                   QAT      NaN  13.00000     1
                                   SAU      NaN  23.00000     1
                                   SDN      NaN  614.00000    1
WHOSIS_000014   2019       NaN     NaN      NaN  22.70000     1
Length: 9926, dtype: int64

<div class="alert alert-info">

* after looking at the 3 unique GHO Codes:
    * They were all genderless stats
    * looking at the value counts of gender, Male and Female do NOT equal Both Sex
    
#### Impute with Both Sex (BTSX) since the responses affect both male and female and will not disrupt the ratio

In [21]:
# Fill NaN with Both Sex (BTSX)
leam_df['SEX'] = leam_df['SEX'].fillna('BTSX')

In [22]:
# Check Work
leam_df.SEX.isnull().sum()

0

In [23]:
# Show Null Count by Column in descending order
leam_df.isnull().sum().sort_values(ascending=False)

COUNTRY    3403
REGION      798
GHO           0
YEAR          0
SEX           0
Numeric       0
dtype: int64

<div class="alert alert-warning">

#### Country

In [24]:
# Show Null Count for COUNTRY
leam_df.COUNTRY.isnull().sum()

3403

In [25]:
# Show unique number of Countries
leam_df.COUNTRY.nunique()

195

In [26]:
# Display Null percentage of entire DataFrame for COUNTRY
print(f'Country Null is {round(leam_df.COUNTRY.isnull().sum() / leam_df.shape[0] *100, 2)} % of entire DataFrame')

Country Null is 2.78 % of entire DataFrame


In [27]:
# List all unique Country values
leam_df.COUNTRY.unique()

array(['SLE', 'SSD', 'CAF', 'SOM', 'AFG', 'TCD', 'NGA', 'GNB', 'RWA',
       'GIN', 'MRT', 'LBR', 'GMB', 'ETH', 'TLS', 'NER', 'TZA', 'MLI',
       'CIV', 'BDI', 'DJI', 'KEN', 'COG', 'LSO', 'ERI', 'MWI', 'SWZ',
       'COD', 'CMR', 'ZWE', nan, 'AGO', 'ALB', 'ARE', 'ARG', 'ARM', 'ATG',
       'AUS', 'AUT', 'AZE', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR',
       'BHS', 'MOZ', 'HTI', 'GNQ', 'SEN', 'MDG', 'COM', 'TGO', 'GHA',
       'UGA', 'BIH', 'BLR', 'BLZ', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN',
       'BWA', 'CAN', 'CHE', 'CHL', 'CHN', 'COL', 'CPV', 'CRI', 'CUB',
       'CYP', 'CZE', 'DEU', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ESP',
       'EST', 'FIN', 'FJI', 'FRA', 'FSM', 'GAB', 'GBR', 'GEO', 'GRC',
       'GRD', 'GTM', 'GUY', 'HND', 'HRV', 'HUN', 'IDN', 'IND', 'IRL',
       'IRN', 'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'SDN', 'JOR', 'JPN',
       'KAZ', 'KGZ', 'KHM', 'KIR', 'KOR', 'KWT', 'LAO', 'LBN', 'LBY',
       'LCA', 'LKA', 'LTU', 'LUX', 'LVA', 'MAR', 'MDA', 'MDV', 'MEX',
       'MKD', '

In [28]:
# Create mask df for Null Country values
country_df = leam_df['COUNTRY'].isnull()

In [29]:
# Take a look
leam_df[country_df]

,GHO,YEAR,REGION,COUNTRY,SEX,Numeric
113,WHOSIS_000004,2001,AFR,NaN,BTSX,397.49860
114,WHOSIS_000004,2002,AFR,NaN,MLE,422.82470
115,WHOSIS_000004,2002,AFR,NaN,FMLE,377.24120
116,WHOSIS_000004,2002,AFR,NaN,BTSX,399.60170
117,WHOSIS_000004,2003,AFR,NaN,MLE,421.73250
...,...,...,...,...,...,...
121628,MDG_0000000003,2015-2020,GLOBAL,NaN,FMLE,42.52991
121912,MDG_0000000025,2001-2007,GLOBAL,NaN,BTSX,64.12033
121913,MDG_0000000025,2015-2021,GLOBAL,NaN,BTSX,83.56604
122198,MDG_0000000025,2008-2014,GLOBAL,NaN,BTSX,76.85271


In [30]:
# List unique Region values in the Null country_df
leam_df[country_df].REGION.unique()

array(['AFR', 'AMR', 'SEAR', 'EUR', 'EMR', 'WPR', 'GLOBAL', nan],
      dtype=object)

In [31]:
# Show the Region value counts under the Null country_df
leam_df[country_df].REGION.value_counts()

AFR       375
AMR       375
SEAR      375
EUR       375
EMR       375
WPR       375
GLOBAL    375
Name: REGION, dtype: int64

In [32]:
# Show the Region value counts under the leam_df
leam_df.REGION.value_counts()

EUR       31630
AFR       29293
AMR       23252
WPR       16356
EMR       13461
SEAR       7035
GLOBAL      375
Name: REGION, dtype: int64

<div class="alert alert-info">

* There are 3403 Null values for country
    * That's less than 3% of the DataFrame
    * looking at the Null country_df Region values, Country is NaN for Global only stats
    * Looking at the leam_df, there is the same count of GLOBAL values meaning there are no other entries for GLOBAL REGION under COUNTRY besides NaN
    
#### Impute country NaN with GLOBAL where REGION equals GLOBAL 
#### DROP remaining NaN in country column

In [33]:
# IMPUTE NaN for [COUNTRY] with 'GLOBAL' if [REGION] == 'GLOBAL'
missing_mask = leam_df['COUNTRY'].isna()
mapping_dict = dict({'GLOBAL': 'GLOBAL'})
leam_df.loc[missing_mask, 'COUNTRY'] = leam_df.loc[missing_mask, 'REGION'].map(mapping_dict)
#leam_df['COUNTRY'] = leam_df['COUNTRY'].fillna('UNK')

In [34]:
# Create another mask for Null Country
country_df = leam_df['COUNTRY'].isnull()

In [35]:
# Check work
leam_df[country_df]

,GHO,YEAR,REGION,COUNTRY,SEX,Numeric
113,WHOSIS_000004,2001,AFR,NaN,BTSX,397.49860
114,WHOSIS_000004,2002,AFR,NaN,MLE,422.82470
115,WHOSIS_000004,2002,AFR,NaN,FMLE,377.24120
116,WHOSIS_000004,2002,AFR,NaN,BTSX,399.60170
117,WHOSIS_000004,2003,AFR,NaN,MLE,421.73250
...,...,...,...,...,...,...
40465,MDG_0000000007,2018,WPR,NaN,BTSX,11.76156
40466,MDG_0000000007,2018,WPR,NaN,FMLE,10.74777
40467,MDG_0000000007,2018,WPR,NaN,MLE,12.71265
40468,MDG_0000000007,2019,WPR,NaN,BTSX,11.23974


In [36]:
# ensure GLOBAL no longer has NULL country values
leam_df[country_df].REGION.unique()

array(['AFR', 'AMR', 'SEAR', 'EUR', 'EMR', 'WPR', nan], dtype=object)

In [37]:
# Show Null Count by Column in descending order
leam_df.isnull().sum().sort_values(ascending=False)

COUNTRY    3028
REGION      798
GHO           0
YEAR          0
SEX           0
Numeric       0
dtype: int64

In [38]:
# Check Row Count
leam_df.shape[0]

122200

In [39]:
# Check effect of Dropping all NaN in DataFrame
leam_df.dropna().T

,0,1,2,3,4,5,6,7,8,9,...,122190,122191,122192,122193,122194,122195,122196,122197,122198,122199
GHO,MDG_0000000026,MDG_0000000026,MDG_0000000026,MDG_0000000026,MDG_0000000026,MDG_0000000026,MDG_0000000026,MDG_0000000026,MDG_0000000026,MDG_0000000026,...,MDG_0000000007,MDG_0000000007,MDG_0000000007,MDG_0000000007,MDG_0000000007,MDG_0000000007,MDG_0000000007,MDG_0000000007,MDG_0000000025,MDG_0000000003
YEAR,2004,2000,2005,2001,2002,2003,2004,2004,2000,2010,...,2010,2010,2013,2013,2016,2016,2018,2019,2008-2014,2000-2005
REGION,AFR,AFR,AFR,AFR,AFR,AFR,EMR,AFR,EMR,EMR,...,AFR,AFR,AFR,AFR,AFR,AFR,AFR,AFR,GLOBAL,GLOBAL
COUNTRY,SLE,SSD,SLE,SSD,CAF,CAF,SOM,CAF,AFG,SOM,...,ZWE,ZWE,ZWE,ZWE,ZWE,ZWE,ZWE,ZWE,GLOBAL,GLOBAL
SEX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,...,BTSX,MLE,BTSX,MLE,BTSX,MLE,FMLE,BTSX,BTSX,FMLE
Numeric,1850.0,1730.0,1760.0,1690.0,1280.0,1250.0,1080.0,1230.0,1450.0,985.0,...,86.42749,92.0966,66.29513,71.38609,58.66147,63.56478,49.82213,54.23561,76.85271,53.04571


In [40]:
# Do the Math
leam_df.shape[0] - leam_df.dropna().shape[0]

3048

In [41]:
# Check Percentage of dropping remainig NULL
print(f'Remaining Null/NaN equals {round(leam_df.dropna().shape[0] / leam_df.shape[0] - 1, 2) * -100} % of entries')

Remaining Null/NaN equals 2.0 % of entries


In [42]:
# DROP remaining Null
leam_df = leam_df.dropna()

In [43]:
# Take a look
leam_df

,GHO,YEAR,REGION,COUNTRY,SEX,Numeric
0,MDG_0000000026,2004,AFR,SLE,BTSX,1850.00000
1,MDG_0000000026,2000,AFR,SSD,BTSX,1730.00000
2,MDG_0000000026,2005,AFR,SLE,BTSX,1760.00000
3,MDG_0000000026,2001,AFR,SSD,BTSX,1690.00000
4,MDG_0000000026,2002,AFR,CAF,BTSX,1280.00000
...,...,...,...,...,...,...
122195,MDG_0000000007,2016,AFR,ZWE,MLE,63.56478
122196,MDG_0000000007,2018,AFR,ZWE,FMLE,49.82213
122197,MDG_0000000007,2019,AFR,ZWE,BTSX,54.23561
122198,MDG_0000000025,2008-2014,GLOBAL,GLOBAL,BTSX,76.85271


In [44]:
# Check Work
leam_df.isnull().sum()

GHO        0
YEAR       0
REGION     0
COUNTRY    0
SEX        0
Numeric    0
dtype: int64

<div class="alert alert-warning">

# PIVOT

In [45]:
# Create df to check for duplicate records using columns that would create a unique index
dupe = leam_df[['YEAR','COUNTRY','SEX', 'GHO', 'Numeric']].duplicated(keep=False)

In [46]:
# Take a look (should be empty)
leam_df[dupe].T

""
GHO
YEAR
REGION
COUNTRY
SEX
Numeric


In [47]:
# Create pivot table
leam_pivot = leam_df.pivot_table(index=['YEAR','COUNTRY','SEX'], columns='GHO', values='Numeric')

In [48]:
# Take a look
leam_pivot

GHO                     MDG_0000000001  MDG_0000000003  MDG_0000000007  \
YEAR      COUNTRY SEX                                                    
1932      GHA     BTSX             NaN             NaN       366.67647   
1934      BRA     BTSX       195.73370             NaN       267.34124   
1935      BRA     BTSX       193.81456             NaN       264.58854   
1936      BRA     BTSX       191.64590             NaN       261.47838   
1937      BRA     BTSX       189.35907             NaN       258.19967   
...                                ...             ...             ...   
2020      ZWE     FMLE             NaN             NaN        49.02382   
                  MLE              NaN             NaN        58.52750   
2020-2021 VNM     BTSX             NaN             NaN             NaN   
2021      FJI     BTSX             NaN             NaN             NaN   
          NER     BTSX             NaN             NaN             NaN   

GHO                     MDG_0000000025  MDG_0000000026  WHOSIS_000001  \
YEAR      COUNTRY SEX                                                   
1932      GHA     BTSX             NaN             NaN            NaN   
1934      BRA     BTSX             NaN             NaN            NaN   
1935      BRA     BTSX             NaN             NaN            NaN   
1936      BRA     BTSX             NaN             NaN            NaN   
1937      BRA     BTSX             NaN             NaN            NaN   
...                                ...             ...            ...   
2020      ZWE     FMLE             NaN             NaN            NaN   
                  MLE              NaN             NaN            NaN   
2020-2021 VNM     BTSX            96.1             NaN            NaN   
2021      FJI     BTSX            99.8             NaN            NaN   
          NER     BTSX            43.6             NaN            NaN   

GHO                     WHOSIS_000002  WHOSIS_000003  WHOSIS_000004  \
YEAR      COUNTRY SEX                                                 
1932      GHA     BTSX            NaN            NaN            NaN   
1934      BRA     BTSX            NaN            NaN            NaN   
1935      BRA     BTSX            NaN            NaN            NaN   
1936      BRA     BTSX            NaN            NaN            NaN   
1937      BRA     BTSX            NaN            NaN            NaN   
...                               ...            ...            ...   
2020      ZWE     FMLE            NaN            NaN            NaN   
                  MLE             NaN            NaN            NaN   
2020-2021 VNM     BTSX            NaN            NaN            NaN   
2021      FJI     BTSX            NaN            NaN            NaN   
          NER     BTSX            NaN            NaN            NaN   

GHO                     WHOSIS_000007  WHOSIS_000014  WHOSIS_000015  \
YEAR      COUNTRY SEX                                                 
1932      GHA     BTSX            NaN            NaN            NaN   
1934      BRA     BTSX            NaN            NaN            NaN   
1935      BRA     BTSX            NaN            NaN            NaN   
1936      BRA     BTSX            NaN            NaN            NaN   
1937      BRA     BTSX            NaN            NaN            NaN   
...                               ...            ...            ...   
2020      ZWE     FMLE            NaN            NaN            NaN   
                  MLE             NaN            NaN            NaN   
2020-2021 VNM     BTSX            NaN            NaN            NaN   
2021      FJI     BTSX            NaN            NaN            NaN   
          NER     BTSX            NaN            NaN            NaN   

GHO                     WHOSIS_000016  
YEAR      COUNTRY SEX                  
1932      GHA     BTSX            NaN  
1934      BRA     BTSX            NaN  
1935      BRA     BTSX            NaN  
1936      BRA     BTSX            NaN  

In [49]:
# assign pivot to df and reset index
leam_df = leam_pivot.reset_index()

In [50]:
# Check Work
leam_df.T

,0,1,2,3,4,5,6,7,8,9,...,63056,63057,63058,63059,63060,63061,63062,63063,63064,63065
GHO,,,,,,,,,,,,,,,,,,,,,
YEAR,1932,1934,1935,1936,1937,1937,1937,1938,1938,1938,...,2020,2020,2020,2020,2020,2020,2020,2020-2021,2021,2021
COUNTRY,GHA,BRA,BRA,BRA,BRA,GHA,GIN,BRA,GHA,GIN,...,ZAF,ZMB,ZMB,ZMB,ZWE,ZWE,ZWE,VNM,FJI,NER
SEX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,...,MLE,BTSX,FMLE,MLE,BTSX,FMLE,MLE,BTSX,BTSX,BTSX
MDG_0000000001,NaN,195.7337,193.81456,191.6459,189.35907,199.09533,NaN,187.47564,194.97678,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDG_0000000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDG_0000000007,366.67647,267.34124,264.58854,261.47838,258.19967,333.37158,352.70782,255.49932,326.85801,353.40746,...,34.77191,61.37835,56.46017,66.14539,53.88819,49.02382,58.5275,NaN,NaN,NaN
MDG_0000000025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.1,99.8,43.6
MDG_0000000026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
WHOSIS_000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# check null
leam_df.isnull().sum().sort_values()

GHO
YEAR                  0
COUNTRY               0
SEX                   0
MDG_0000000007    27540
MDG_0000000001    27691
WHOSIS_000003     53604
WHOSIS_000004     53682
WHOSIS_000016     56990
WHOSIS_000014     59183
MDG_0000000026    59754
MDG_0000000003    60133
WHOSIS_000001     60858
WHOSIS_000002     60858
WHOSIS_000007     60858
WHOSIS_000015     60858
MDG_0000000025    60985
dtype: int64

In [52]:
# Impute Null with 0
leam_df = leam_df.fillna(0)

In [53]:
# Check work
leam_df.head()

GHO,YEAR,COUNTRY,SEX,MDG_0000000001,MDG_0000000003,MDG_0000000007,MDG_0000000025,MDG_0000000026,WHOSIS_000001,WHOSIS_000002,WHOSIS_000003,WHOSIS_000004,WHOSIS_000007,WHOSIS_000014,WHOSIS_000015,WHOSIS_000016
0,1932,GHA,BTSX,0.00000,0.0,366.67647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1934,BRA,BTSX,195.73370,0.0,267.34124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1935,BRA,BTSX,193.81456,0.0,264.58854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1936,BRA,BTSX,191.64590,0.0,261.47838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1937,BRA,BTSX,189.35907,0.0,258.19967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
# check new record count after pivot
leam_df.shape[0]

63066

<div class="alert alert-warning">

# Handle YEAR Column

In [55]:
# List unique year count
leam_df.YEAR.nunique()

210

In [56]:
# List year value counts
leam_df.YEAR.value_counts()

2010         588
2007         588
2004         588
2003         588
2002         588
            ... 
2005-2010      1
2006-2007      1
2012-2014      1
2008-2014      1
1932           1
Name: YEAR, Length: 210, dtype: int64

In [57]:
# List unique year values
leam_df.YEAR.unique()

array([1932, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943,
       1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954,
       1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965,
       1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976,
       1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987,
       1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998,
       1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       '1933', '1934', '1935', '1936', '1940', '1941', '1942', '1943',
       '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951',
       '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967',
       '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975',
       '1976', '1977', '1978', '1979', '1980', '1981', '1982'

In [58]:
# Create mask with year values that contain "-" to capture multiple entries under one value
year_df = leam_df['YEAR'].str.contains('-', case=False, na=False)

In [59]:
# Check Work
year_df

0        False
1        False
2        False
3        False
4        False
         ...  
63061    False
63062    False
63063     True
63064    False
63065    False
Name: YEAR, Length: 63066, dtype: bool

In [60]:
# Check Row Count of mask
leam_df[year_df].shape[0]

148

In [61]:
# Take a look
leam_df[year_df]

GHO,YEAR,COUNTRY,SEX,MDG_0000000001,MDG_0000000003,MDG_0000000007,MDG_0000000025,MDG_0000000026,WHOSIS_000001,WHOSIS_000002,WHOSIS_000003,WHOSIS_000004,WHOSIS_000007,WHOSIS_000014,WHOSIS_000015,WHOSIS_000016
50985,1999-2000,BGD,BTSX,0.0,0.0,0.0,12.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50986,1999-2000,CRI,BTSX,0.0,0.0,0.0,96.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50987,1999-2000,ITA,BTSX,0.0,0.0,0.0,99.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50988,1999-2000,LBN,BTSX,0.0,0.0,0.0,99.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50989,1999-2000,LBR,BTSX,0.0,0.0,0.0,50.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62655,2019-2020,RWA,BTSX,0.0,0.0,0.0,94.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62656,2019-2020,TUV,BTSX,0.0,0.0,0.0,99.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62657,2019-2020,WSM,BTSX,0.0,0.0,0.0,88.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62658,2019-2021,IND,BTSX,0.0,0.0,0.0,89.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
# delete all rows where column [YEAR] contains '-'
indexYear = leam_df[leam_df['YEAR'].str.contains('-', case=False, na=False)].index
leam_df.drop(indexYear, inplace=True)

In [63]:
# List Year values
leam_df.YEAR.unique()

array([1932, 1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943,
       1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954,
       1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965,
       1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976,
       1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987,
       1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998,
       1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009,
       2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       '1933', '1934', '1935', '1936', '1940', '1941', '1942', '1943',
       '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951',
       '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967',
       '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975',
       '1976', '1977', '1978', '1979', '1980', '1981', '1982'

In [64]:
# Check Row Count
leam_df.shape[0]

62918

<div class="alert alert-info">

# Feature Engineer
* Replace Target Numeric Column that was lost during pivot
* Original Numeric Column held both Life Expectancy in Years and Percentage values
* Split those values properly into Two Separate Features:
    * Life Expectancy in Years (ley) Coloumn will equal mean of all Life Expectancy feature columns
    * Percentage (prcnt) Column will equal mean of all percentage feature columns

<div class="alert alert-warning">

#### Life Expactancy in Years

* 'WHOSIS_000001': 'Life expectancy at birth (years)',
* 'WHOSIS_000002': 'Healthy life expectancy (HALE) at birth (years)',
* 'WHOSIS_000007': 'Healthy life expectancy (HALE) at age 60 (years)',
* 'WHOSIS_000015': 'Life expectancy at age 60 (years)',

In [65]:
leam_df['life_expectancy'] = leam_df[['WHOSIS_000001','WHOSIS_000002','WHOSIS_000007','WHOSIS_000015']].mean(axis=1)

In [66]:
leam_df

GHO,YEAR,COUNTRY,SEX,MDG_0000000001,MDG_0000000003,MDG_0000000007,MDG_0000000025,MDG_0000000026,WHOSIS_000001,WHOSIS_000002,WHOSIS_000003,WHOSIS_000004,WHOSIS_000007,WHOSIS_000014,WHOSIS_000015,WHOSIS_000016,life_expectancy
0,1932,GHA,BTSX,0.00000,0.0,366.67647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1934,BRA,BTSX,195.73370,0.0,267.34124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1935,BRA,BTSX,193.81456,0.0,264.58854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1936,BRA,BTSX,191.64590,0.0,261.47838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1937,BRA,BTSX,189.35907,0.0,258.19967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63060,2020,ZWE,BTSX,0.00000,0.0,53.88819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63061,2020,ZWE,FMLE,0.00000,0.0,49.02382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63062,2020,ZWE,MLE,0.00000,0.0,58.52750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63064,2021,FJI,BTSX,0.00000,0.0,0.00000,99.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<div class="alert alert-danger">

#### IS THERE A PROBLEM WITH THE WAY THE MEAN IS BEING CALCULATED?

In [67]:
# Null/MIN/MAX of MEAN Life Expectancy in Years (ley)
leam_df.life_expectancy.isnull().sum(), leam_df.life_expectancy.min(), leam_df.life_expectancy.max()

(0, 0.0, 53.206227500000004)

In [68]:
# MIN/MAX of Life expectancy at birth (years) 
leam_df.WHOSIS_000001.min(), leam_df.WHOSIS_000001.max()

(0.0, 86.93782)

In [69]:
# MIN/MAX of Healthy life expectancy (HALE) at birth (years)
leam_df.WHOSIS_000002.min(), leam_df.WHOSIS_000002.max()

(0.0, 75.48193)

In [70]:
# MIN/MAX of Healthy life expectancy (HALE) at age 60 (years)
leam_df.WHOSIS_000007.min(), leam_df.WHOSIS_000007.max()

(0.0, 21.84938)

In [71]:
# MIN/MAX of Life expectancy at age 60 (years)
leam_df.WHOSIS_000015.min(), leam_df.WHOSIS_000015.max()

(0.0, 28.55578)

In [72]:
# MEAN of all MAX
(leam_df.WHOSIS_000001.max() + leam_df.WHOSIS_000002.max() + leam_df.WHOSIS_000007.max() + leam_df.WHOSIS_000015.max()) / 4

53.206227500000004

In [73]:
# Including 0 or null into the mean
(25 + 0 + 0 + 25)/4

12.5

In [74]:
# Same input calculating mean without inclusion of 0
(25 + 0 + 0 + 25)/2

25.0

<div class="alert alert-info">

## YES

* On the surface it appears that the overall aggregate mean should be more in line with each of its contributors
* At the end of the day, this feature will have a much lower mean because it is diluted by the number of imputed 0 entries for NaN
* Feeding the Machine Learning Algorythm imputed 0 for NaN is not optimal
    * But, there are so many NaN in the data set that imputing mean will give an inaccurate representation of the data

<div class="alert alert-danger">
    
### In the interst of time and project completion, I will move forward with imputing mean for all NaN so that the Machine Learning Algorythm will have significant data to compute...  WITH the explicit knowledge that this is not an optimal decision and will not produce a realistically viable model even if it appears to perform well. 

<div class="alert alert-warning">

#### Percentage Rates

* 'WHOSIS_000003': 'Neonatal mortality rate (0 to 27 days) per 1000 live births)',
* 'WHOSIS_000004': 'Adult mortality rate (probability of dying between 15 and 60 years per 1000 population)',
* 'WHOSIS_000014': 'Stillbirth rate (per 1000 total births)',
* 'WHOSIS_000016': 'Mortality rate among children ages 5 to 9 years (per 1000 children aged 5)',
*     
* 'MDG_0000000001': 'Infant mortality rate (between birth and 11 months per 1000 live births)',
* 'MDG_0000000003': 'Adolescent birth rate (per 1000 women)',
* 'MDG_0000000007': 'Under-five mortality rate (per 1000 live births)',
* 'MDG_0000000025': 'Births attended by skilled health personnel (%)',
* 'MDG_0000000026': 'Maternal mortality ratio (per 100 000 live births)',

<div class="alert alert-danger">

#### IS THERE A PROBLEM WITH THE WAY THE MEAN IS BEING CALCULATED?

In [75]:
# MIN/MAX of Neonatal mortality rate (0 to 27 days) per 1000 live births
leam_df.WHOSIS_000003.min(), leam_df.WHOSIS_000003.max(), leam_df.WHOSIS_000003.max() / 1_000

(0.0, 118.32012, 0.11832012)

In [76]:
# MIN/MAX of Adult mortality rate (probability of dying between 15 and 60 years per 1000 population)
leam_df.WHOSIS_000004.min(), leam_df.WHOSIS_000004.max(), leam_df.WHOSIS_000004.max() / 1_000

(0.0, 769.2899, 0.7692899)

In [77]:
# MIN/MAX of Stillbirth rate (per 1000 total births)
leam_df.WHOSIS_000014.min(), leam_df.WHOSIS_000014.max(), leam_df.WHOSIS_000014.max() / 1_000

(0.0, 44.79614, 0.04479614)

In [78]:
# MIN/MAX of Mortality rate among children ages 5 to 9 years (per 1000 children aged 5)
leam_df.WHOSIS_000016.min(), leam_df.WHOSIS_000016.max(), leam_df.WHOSIS_000016.max() / 1_000

(0.0, 150.5262, 0.1505262)

In [79]:
# MIN/MAX of Infant mortality rate (between birth and 11 months per 1000 live births)
leam_df.MDG_0000000001.min(), leam_df.MDG_0000000001.max(), leam_df.MDG_0000000001.max() / 1_000

(0.0, 296.13938, 0.29613938)

In [80]:
# MIN/MAX of Adolescent birth rate (per 1000 women)
leam_df.MDG_0000000003.min(), leam_df.MDG_0000000003.max(), leam_df.MDG_0000000003.max() / 1_000

(0.0, 229.0, 0.229)

In [81]:
# MIN/MAX of Under-five mortality rate (per 1000 live births)
leam_df.MDG_0000000007.min(), leam_df.MDG_0000000007.max(), leam_df.MDG_0000000007.max() / 1_000

(0.0, 569.09668, 0.56909668)

In [82]:
# MIN/MAX of Births attended by skilled health personnel (%)
leam_df.MDG_0000000025.min(), leam_df.MDG_0000000025.max(), leam_df.MDG_0000000025.max() / 100

(0.0, 100.0, 1.0)

In [83]:
# MIN/MAX of Maternal mortality ratio (per 100 000 live births)
leam_df.MDG_0000000026.min(), leam_df.MDG_0000000026.max(), leam_df.MDG_0000000026.max()/100_000

(0.0, 2480.0, 0.0248)

<div class="alert alert-info">

## YES
* After checking the MIN/MAX and reading the description of each GHO Code, I discovered they are measured against different ratio's of *100, *1_000, and *100_000
* Converting Percentages would absolutely sufficient to rectify this balancing issue
    * However... there are a multitude of NaN entries that have been imputed to 0 which is causing inaccurate noise in the individual features to be fed to the model
    * Even if we were to remove the 0 count from the mean calculation to produce a "clean" mean aggregate, the individual features would still be tainted with too many 0 data imputations to be reliably and realistically relevant
    
<div class="alert alert-danger">
    
### In the interst of time and project completion, I will move forward with imputing mean for all NaN so that the Machine Learning Algorythm will have significant data to compute...  WITH the explicit knowledge that this is not an optimal decision and will not produce a realistically viable model even if it appears to perform well. 

<div class="alert alert-warning">

#### Convert Percentages in order to be on same ratios

In [84]:
leam_df["WHOSIS_000003_prcnt"] = leam_df["WHOSIS_000003"] / 1_000

In [85]:
leam_df["WHOSIS_000004_prcnt"] = leam_df["WHOSIS_000004"] / 1_000

In [86]:
leam_df["WHOSIS_000014_prcnt"] = leam_df["WHOSIS_000014"] / 1_000

In [87]:
leam_df["WHOSIS_000016_prcnt"] = leam_df["WHOSIS_000016"] / 1_000

In [88]:
leam_df["MDG_0000000001_prcnt"] = leam_df["MDG_0000000001"] / 1_000

In [89]:
leam_df["MDG_0000000003_prcnt"] = leam_df["MDG_0000000003"] / 1_000

In [90]:
leam_df["MDG_0000000007_prcnt"] = leam_df["MDG_0000000007"] / 1_000

In [91]:
leam_df["MDG_0000000025_prcnt"] = leam_df["MDG_0000000025"] / 100

In [92]:
leam_df["MDG_0000000026_prcnt"] = leam_df["MDG_0000000026"] / 100_000

In [93]:
leam_df['prcnt'] = leam_df[['WHOSIS_000003_prcnt','WHOSIS_000004_prcnt','WHOSIS_000014_prcnt','WHOSIS_000016_prcnt',
                           'MDG_0000000001_prcnt','MDG_0000000003_prcnt','MDG_0000000007_prcnt','MDG_0000000025_prcnt','MDG_0000000026_prcnt'
                          ]].mean(axis=1)

In [94]:
# Drop original columns after conversion to proper percentage ratio
leam_df.drop(columns=['WHOSIS_000003','WHOSIS_000004','WHOSIS_000014','WHOSIS_000016',
                      'MDG_0000000001','MDG_0000000003','MDG_0000000007','MDG_0000000025','MDG_0000000026'])

GHO,YEAR,COUNTRY,SEX,WHOSIS_000001,WHOSIS_000002,WHOSIS_000007,WHOSIS_000015,life_expectancy,WHOSIS_000003_prcnt,WHOSIS_000004_prcnt,WHOSIS_000014_prcnt,WHOSIS_000016_prcnt,MDG_0000000001_prcnt,MDG_0000000003_prcnt,MDG_0000000007_prcnt,MDG_0000000025_prcnt,MDG_0000000026_prcnt,prcnt
0,1932,GHA,BTSX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.366676,0.000,0.0,0.040742
1,1934,BRA,BTSX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.195734,0.0,0.267341,0.000,0.0,0.051453
2,1935,BRA,BTSX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.193815,0.0,0.264589,0.000,0.0,0.050934
3,1936,BRA,BTSX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.191646,0.0,0.261478,0.000,0.0,0.050347
4,1937,BRA,BTSX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.189359,0.0,0.258200,0.000,0.0,0.049729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63060,2020,ZWE,BTSX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.053888,0.000,0.0,0.005988
63061,2020,ZWE,FMLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.049024,0.000,0.0,0.005447
63062,2020,ZWE,MLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.058528,0.000,0.0,0.006503
63064,2021,FJI,BTSX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.998,0.0,0.110889


In [95]:
leam_df['prcnt'].min(), leam_df['prcnt'].max(), leam_df['prcnt'].mean()

(0.0, 0.18065488444444444, 0.015600672072574215)

In [96]:
leam_df.T

,0,1,2,3,4,5,6,7,8,9,...,63055,63056,63057,63058,63059,63060,63061,63062,63064,63065
GHO,,,,,,,,,,,,,,,,,,,,,
YEAR,1932,1934,1935,1936,1937,1937,1937,1938,1938,1938,...,2020,2020,2020,2020,2020,2020,2020,2020,2021,2021
COUNTRY,GHA,BRA,BRA,BRA,BRA,GHA,GIN,BRA,GHA,GIN,...,ZAF,ZAF,ZMB,ZMB,ZMB,ZWE,ZWE,ZWE,FJI,NER
SEX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,BTSX,...,FMLE,MLE,BTSX,FMLE,MLE,BTSX,FMLE,MLE,BTSX,BTSX
MDG_0000000001,0.0,195.7337,193.81456,191.6459,189.35907,199.09533,0.0,187.47564,194.97678,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MDG_0000000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MDG_0000000007,366.67647,267.34124,264.58854,261.47838,258.19967,333.37158,352.70782,255.49932,326.85801,353.40746,...,29.52349,34.77191,61.37835,56.46017,66.14539,53.88819,49.02382,58.5275,0.0,0.0
MDG_0000000025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.8,43.6
MDG_0000000026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WHOSIS_000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<div class="alert alert-danger">

# <h><i><u><font size="1000"><center>FULL STOP</center></font></u></i></h>
    
* Data was unusable, too many features with too little data entries resulting in massive NaN count
* Imputing data grossly altered integrity

## SCALE BACK PROJECT with less features 

<div class="alert alert-success">

# Checkpoint #1

1. a brief (one-sentence) description of your project
* Predict future Life Expectancy or Healthy Life Expectancy

2. a link to the data source
* **GHO Selections:** https://apps.who.int/gho/athena/api/GHO
* **Homepage:** https://www.who.int/data/gho/info/athena-api-examples
* **CSV Data Pull:** https://apps.who.int/gho/athena/api/GHO/WHOSIS_000001,WHOSIS_000002,WHOSIS_000003,WHOSIS_000004,WHOSIS_000005,WHOSIS_000006,WHOSIS_000007,WHOSIS_000008,WHOSIS_000009,WHOSIS_000010,WHOSIS_000012,WHOSIS_000013,WHOSIS_000014,WHOSIS_000015,WHOSIS_000016?format=csv 

3. your target variable
* Numeric (Expected Years of continued Life)

4. what one observation represents
* Numeric for one year in specified country according to category of life expectancy
    * 'WHOSIS_000001': 'Life expectancy at birth (years)', 
    * 'WHOSIS_000002': 'Healthy life expectancy (HALE) at birth (years)', 
    * 'WHOSIS_000007': 'Healthy life expectancy (HALE) at age 60 (years)', 
    * 'WHOSIS_000015': 'Life expectancy at age 60 (years)', 


<div class="alert alert-danger">

# Sometimes it is faster to read the unreadable data than it is to read about how to make the data readable!